In [122]:
print('Mai kehyaa Namaste !!!')

Mai kehyaa Namaste !!!


In [ ]:
import dropbox
import sqlite3
import pandas as pd
import tempfile
import os

# ======== CONFIG ========
ACCESS_TOKEN = os.getenv("DROPBOX_TOKEN")
DROPBOX_FILE_PATH = "/Apps/HIS_A_create/healthcare-data/HIS_A/patients.db"
TABLE_NAME = "patients"   # Change this to your table name if different
# ========================

# Step 1: Connect to Dropbox
dbx = dropbox.Dropbox(ACCESS_TOKEN)

# Step 2: Download to temporary file
with tempfile.NamedTemporaryFile(delete=False, suffix=".db") as tmp_db:
    dbx.files_download_to_file(tmp_db.name, DROPBOX_FILE_PATH)
    temp_path = tmp_db.name

# Step 3: Verify file integrity
file_size = os.path.getsize(temp_path)
with open(temp_path, "rb") as f:
    header = f.read(16)

if file_size == 0 or not header.startswith(b"SQLite format"):
    raise ValueError("Downloaded file is not a valid SQLite database!")

print(f"✅ Downloaded {DROPBOX_FILE_PATH} ({file_size} bytes)")

# Step 4: Load data into DataFrame
conn = sqlite3.connect(temp_path)
his_a_patients_dataframe = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", conn)
conn.close()

# Step 5: Preview
print(his_a_patients_dataframe.head())


Unable to refresh access token without                 refresh token and app key


AuthError: AuthError('b11a1da4b4884bdf8e180d5710670d5d', AuthError('expired_access_token', None))

# As there are problems while working with access token, we will download it with request, public access


# First we will deal witrh patients data

# HIS_A patients

In [ ]:
import requests
import sqlite3
import pandas as pd
import tempfile

dropbox_url = "https://www.dropbox.com/scl/fi/bpyevimiyyfyp6anzdo74/patients.db?rlkey=nnbonek6cqzmr5rs9glu02jc0&dl=1"

with tempfile.NamedTemporaryFile(delete=False, suffix=".db") as tmp_db:
    response = requests.get(dropbox_url)
    response.raise_for_status()  
    tmp_db.write(response.content)
    tmp_db_path = tmp_db.name

conn = sqlite3.connect(tmp_db_path)

table_names = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", table_names)

his_a_patients_dataframe = pd.read_sql("SELECT * FROM patients", conn)

# Step 5: Check data
print(his_a_patients_dataframe.head())

conn.close()


Tables in DB:        name
0  patients
  patient_id first_name last_name         dob  gender blood_group  height_cm  \
0   81f91df9      Grace   Andrews  1957-07-28    Male         AB+        191   
1   dfd520ae     Sherri     Smith  1994-04-02    Male          A-        161   
2   f6173246       Dana  Phillips  1941-03-10  Female          A-        159   
3   79326454       Paul  Campbell  1941-07-01    Male         AB-        196   
4   59978f9f     Daniel     Jones  2016-03-14    Male         AB-        192   

   weight_kg country        state  ... insurance_id insurance_provider  \
0         93   India  Maharashtra  ...     INS70403     Aarogya Kavach   
1         98   India  Maharashtra  ...     INS34028      ABC Insurance   
2         56   India  Maharashtra  ...     INS81776         XYZ Health   
3        110   India  Maharashtra  ...     INS51123           CarePlus   
4        120   India  Maharashtra  ...     INS68384      ABC Insurance   

  primary_physician medical_history 

In [ ]:
his_a_patients_dataframe.shape

(1200, 27)

In [ ]:
his_a_patients_dataframe.columns

Index(['patient_id', 'first_name', 'last_name', 'dob', 'gender', 'blood_group',
       'height_cm', 'weight_kg', 'country', 'state', 'city', 'postal_code',
       'phone', 'email', 'emergency_contact_name',
       'emergency_contact_relation', 'emergency_contact_phone', 'insurance_id',
       'insurance_provider', 'primary_physician', 'medical_history', 'disease',
       'patient_status', 'severity', 'department_admitted',
       'current_department', 'bed_type'],
      dtype='object')

In [ ]:
his_a_patients_dataframe.head(3)

,patient_id,first_name,last_name,dob,gender,blood_group,height_cm,weight_kg,country,state,...,insurance_id,insurance_provider,primary_physician,medical_history,disease,patient_status,severity,department_admitted,current_department,bed_type
0,81f91df9,Grace,Andrews,1957-07-28,Male,AB+,191,93,India,Maharashtra,...,INS70403,Aarogya Kavach,Dr. Klein,Hypertension,Malaria,Discharged,Moderate,Neurology,Neurology,Severe Ward
1,dfd520ae,Sherri,Smith,1994-04-02,Male,A-,161,98,India,Maharashtra,...,INS34028,ABC Insurance,Dr. Bates,Asthma,Flu,Discharged,Mild,Emergency,Cardiology,Severe Ward
2,f6173246,Dana,Phillips,1941-03-10,Female,A-,159,56,India,Maharashtra,...,INS81776,XYZ Health,Dr. Schultz,Diabetes,Heart Disease,Observation,Critical,Gastroenterology,Cardiology,General


# HIS_B patients

In [ ]:
import requests
import json

dropbox_url_his_b = "https://www.dropbox.com/scl/fi/r1h1j2xqkyg8f6c0ugwt1/patients.json?rlkey=57zx1msyqeqvpgb7qf7p6a7r8&st=88advxco&dl=1"

response = requests.get(dropbox_url_his_b)
response.raise_for_status()  

his_b_patients_json = response.json()

print(type(his_b_patients_json))
print(json.dumps(his_b_patients_json, indent=2)[:500])  


<class 'list'>
[
  {
    "patient_id": "4be67af7",
    "name": {
      "first": "Amanda",
      "last": "Ruiz"
    },
    "dob": "1962-06-01",
    "gender": "Male",
    "location": {
      "country": "India",
      "state": "Maharashtra",
      "city": "Nashik",
      "postal_code": "11573"
    },
    "contact": {
      "phone": "0830521213773",
      "email": "tjones@example.net"
    },
    "emergency_person": {
      "name": "Jonathan Castro",
      "relation": "Sibling",
      "phone": "5926053769571"
    }


In [ ]:
from pprint import pprint
pprint(his_b_patients_json)

[{'bed_type': 'General',
  'contact': {'email': 'tjones@example.net', 'phone': '0830521213773'},
  'current_department': 'Oncology',
  'department_admitted': 'Cardiology',
  'disease': 'Covid-19',
  'dob': '1962-06-01',
  'emergency_person': {'name': 'Jonathan Castro',
                       'phone': '5926053769571',
                       'relation': 'Sibling'},
  'gender': 'Male',
  'insurance': {'id': 'INS62962', 'provider': 'ABC Insurance'},
  'location': {'city': 'Nashik',
               'country': 'India',
               'postal_code': '11573',
               'state': 'Maharashtra'},
  'medical_info': {'allergies': 'Peanuts',
                   'blood_group': 'O+',
                   'bp': '100/72',
                   'chronic_conditions': 'None',
                   'hb': 15.2},
  'name': {'first': 'Amanda', 'last': 'Ruiz'},
  'patient_id': '4be67af7',
  'patient_status': 'Observation',
  'primary_physician': 'Dr. Prince',
  'severity': 'treating'},
 {'bed_type': 'Severe Ward',
 

In [ ]:
pprint(his_b_patients_json[:4])

[{'bed_type': 'General',
  'contact': {'email': 'tjones@example.net', 'phone': '0830521213773'},
  'current_department': 'Oncology',
  'department_admitted': 'Cardiology',
  'disease': 'Covid-19',
  'dob': '1962-06-01',
  'emergency_person': {'name': 'Jonathan Castro',
                       'phone': '5926053769571',
                       'relation': 'Sibling'},
  'gender': 'Male',
  'insurance': {'id': 'INS62962', 'provider': 'ABC Insurance'},
  'location': {'city': 'Nashik',
               'country': 'India',
               'postal_code': '11573',
               'state': 'Maharashtra'},
  'medical_info': {'allergies': 'Peanuts',
                   'blood_group': 'O+',
                   'bp': '100/72',
                   'chronic_conditions': 'None',
                   'hb': 15.2},
  'name': {'first': 'Amanda', 'last': 'Ruiz'},
  'patient_id': '4be67af7',
  'patient_status': 'Observation',
  'primary_physician': 'Dr. Prince',
  'severity': 'treating'},
 {'bed_type': 'Severe Ward',
 

In [ ]:
import pandas as pd

def flatten_dict(d, parent_key="", sep="_"):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

flat_rows = [flatten_dict(rec) for rec in his_b_patients_json]
df_raw = pd.DataFrame(flat_rows)

rename_map = {
    "name_first": "first_name",
    "name_last": "last_name",
    "location_country": "country",
    "location_state": "state",
    "location_city": "city",
    "location_postal_code": "postal_code",
    "contact_phone": "phone",
    "contact_email": "email",
    "emergency_person_name": "emergency_contact_name",
    "emergency_person_relation": "emergency_contact_relation",
    "emergency_person_phone": "emergency_contact_phone",
    "insurance_id": "insurance_id",
    "insurance_provider": "insurance_provider",
    "medical_info_blood_group": "blood_group",
    "medical_info_bp": "bp",
    "medical_info_hb": "hb",
    "medical_info_allergies": "allergies",
    "medical_info_chronic_conditions": "chronic_conditions",
}
df = df_raw.rename(columns=rename_map)

if "medical_history" not in df.columns:
    df["medical_history"] = df.get("chronic_conditions")

if "email" in df.columns:
    df["email"] = df["email"].astype(str).str.lower()

his_a_cols = [
    "patient_id", "first_name", "last_name", "dob", "gender", "blood_group",
    "height_cm", "weight_kg", "country", "state", "city", "postal_code",
    "phone", "email", "emergency_contact_name", "emergency_contact_relation",
    "emergency_contact_phone", "insurance_id", "insurance_provider",
    "primary_physician", "medical_history", "disease", "patient_status",
    "severity", "department_admitted", "current_department", "bed_type"
]
extra_cols = ["bp", "hb", "allergies", "chronic_conditions"]

for col in his_a_cols + extra_cols:
    if col not in df.columns:
        df[col] = None

final_cols = [c for c in his_a_cols + extra_cols if c in df.columns]
his_b_patients_dataframe = df[final_cols].copy()

print(his_b_patients_dataframe.head(3))
print("\nColumns:", his_b_patients_dataframe.columns.tolist())


  patient_id first_name   last_name         dob gender blood_group height_cm  \
0   4be67af7     Amanda        Ruiz  1962-06-01   Male          O+      None   
1   5ce16a82     Justin  Mccullough  1998-03-21   Male          A-      None   
2   f282b25a    Heather         Cox  2018-09-04   Male          B+      None   

  weight_kg country        state  ...        disease patient_status  severity  \
0      None   India  Maharashtra  ...       Covid-19    Observation  treating   
1      None   India  Maharashtra  ...       Fracture    Observation      Mild   
2      None   India  Maharashtra  ...  Heart Disease       Admitted  Critical   

  department_admitted current_department     bed_type      bp    hb allergies  \
0          Cardiology           Oncology      General  100/72  15.2   Peanuts   
1           Neurology   Gastroenterology  Severe Ward  127/83  11.4      Dust   
2           Neurology         Cardiology          ICU  135/71  11.2      None   

  chronic_conditions  
0     

In [ ]:
his_b_patients_dataframe

,patient_id,first_name,last_name,dob,gender,blood_group,height_cm,weight_kg,country,state,...,disease,patient_status,severity,department_admitted,current_department,bed_type,bp,hb,allergies,chronic_conditions
0,4be67af7,Amanda,Ruiz,1962-06-01,Male,O+,None,None,India,Maharashtra,...,Covid-19,Observation,treating,Cardiology,Oncology,General,100/72,15.2,Peanuts,None
1,5ce16a82,Justin,Mccullough,1998-03-21,Male,A-,None,None,India,Maharashtra,...,Fracture,Observation,Mild,Neurology,Gastroenterology,Severe Ward,127/83,11.4,Dust,Asthma
2,f282b25a,Heather,Cox,2018-09-04,Male,B+,None,None,India,Maharashtra,...,Heart Disease,Admitted,Critical,Neurology,Cardiology,ICU,135/71,11.2,None,Asthma
3,5b88c99c,Heather,Wilson,1939-12-20,Female,O+,None,None,India,Maharashtra,...,Flu,Critical,treating,Oncology,Neurology,General,133/64,12.7,Penicillin,Hypertension
4,1edaa1b1,Elizabeth,Anderson,1996-05-31,Male,O-,None,None,India,Maharashtra,...,Heart Disease,Critical,Severe,Emergency,Orthopedics,General,108/71,13.9,Penicillin,Asthma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,ac55e258,Natalie,Fisher,1959-05-06,Female,O+,None,None,India,Maharashtra,...,Heart Disease,Admitted,treating,Oncology,Radiology,Severe Ward,140/68,16.8,None,None
396,a5688a9e,Jacqueline,Thompson,1951-02-03,Female,O-,None,None,India,Maharashtra,...,Flu,Discharged,treating,Orthopedics,Oncology,General,124/83,14.4,None,None
397,b4d1931a,Timothy,Jones,1967-12-27,Male,A-,None,None,India,Maharashtra,...,Fracture,Discharged,Mild,Radiology,Gastroenterology,Severe Ward,129/73,12.7,Penicillin,None
398,4aa3cc33,Elizabeth,Edwards,1968-02-10,Male,O+,None,None,India,Maharashtra,...,Malaria,Discharged,Severe,Oncology,Neurology,General,118/63,16.4,Penicillin,Diabetes


# HIS_C Patients

In [ ]:
import pandas as pd
import requests
from io import StringIO

his_c_csv_url = "https://www.dropbox.com/scl/fi/vpwzzq19qjmigyv2nc73q/patients.csv?rlkey=ncel01bzbqjtgmxnnlnrtpanv&st=g8g35tvx&dl=1"

response = requests.get(his_c_csv_url)
response.raise_for_status()

his_c_patients_dataframe = pd.read_csv(StringIO(response.text))

print(his_c_patients_dataframe.shape, his_c_patients_dataframe.columns)


(500, 26) Index(['patient_id', 'name', 'dob', 'gender', 'blood_group', 'height_cm',
       'weight_kg', 'medical_history', 'allergies', 'insurance_id',
       'insurance_provider', 'primary_physician', 'specialty', 'duration_days',
       'care_instructions', 'time_bound', 'contact_phone', 'contact_email',
       'emergency_person_name', 'emergency_person_relation',
       'emergency_person_phone', 'department_admitted', 'current_department',
       'bed_type', 'patient_status', 'severity'],
      dtype='object')


In [ ]:
from faker import Faker
fake = Faker('en_GB')  
his_c_patients_dataframe['country'] = [fake.country() for _ in range(len(his_c_patients_dataframe))]
his_c_patients_dataframe['state'] = [fake.administrative_unit() for _ in range(len(his_c_patients_dataframe))]
his_c_patients_dataframe['city'] = [fake.city() for _ in range(len(his_c_patients_dataframe))]
his_c_patients_dataframe['postal_code'] = [fake.postcode() for _ in range(len(his_c_patients_dataframe))]


In [ ]:
his_c_patients_dataframe.columns

Index(['HIS', 'patient_id', 'dob', 'gender', 'blood_group', 'height_cm',
       'weight_kg', 'medical_history', 'allergies', 'insurance_id',
       'insurance_provider', 'primary_physician', 'specialty', 'duration_days',
       'care_instructions', 'time_bound', 'phone', 'email',
       'emergency_contact_name', 'emergency_contact_relation',
       'emergency_contact_phone', 'department_admitted', 'current_department',
       'bed_type', 'patient_status', 'severity', 'last_name',
       'emergency_person', 'postal_code', 'state', 'contact', 'city',
       'disease', 'treatment_protocol', 'bp', 'surgery_required', 'hb',
       'location', 'stage_danger', 'followup_date', 'chronic_conditions',
       'first_name', 'country'],
      dtype='object')

In [ ]:
his_c_patients_dataframe.head(3)

,HIS,patient_id,dob,gender,blood_group,height_cm,weight_kg,medical_history,allergies,insurance_id,...,treatment_protocol,bp,surgery_required,hb,location,stage_danger,followup_date,chronic_conditions,first_name,country
0,HIS_C,cd82ede0,1950-02-02,Female,B-,177,81,Hypertension,NaN,INS77698,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Jennifer,Solomon Islands
1,HIS_C,041abc89,1963-02-03,Male,B-,172,51,NaN,Food,INS64622,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Amber,Tokelau
2,HIS_C,296226c1,1980-05-23,Female,B+,193,55,Asthma,Dust,INS10206,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Lance,Sierra Leone


# HIS_D Patients

In [ ]:
import requests
import sqlite3
import pandas as pd
import tempfile

dropbox_url = "https://www.dropbox.com/scl/fi/w3zeur3r9ss5pf85gjp0p/patients.db?rlkey=ze6kdje6qbgs8cvtvp1s6675w&st=gf7kfqtg&dl=1"

with tempfile.NamedTemporaryFile(delete=False, suffix=".db") as tmp_db:
    response = requests.get(dropbox_url)
    response.raise_for_status()
    tmp_db.write(response.content)
    tmp_db_path = tmp_db.name

conn = sqlite3.connect(tmp_db_path)

table_names = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", table_names)

his_d_patients_dataframe = pd.read_sql("SELECT * FROM patients", conn)

print(his_d_patients_dataframe.shape,'\n',his_d_patients_dataframe.columns)

conn.close()


Tables in DB:        name
0  patients
(400, 24) 
 Index(['patient_id', 'name', 'dob', 'gender', 'blood_group', 'height_cm',
       'weight_kg', 'location', 'contact', 'emergency_person', 'insurance_id',
       'insurance_provider', 'primary_physician', 'medical_history', 'disease',
       'patient_status', 'severity', 'department_admitted',
       'current_department', 'bed_type', 'stage_danger', 'surgery_required',
       'treatment_protocol', 'followup_date'],
      dtype='object')


In [ ]:
his_d_patients_dataframe.head(3)

,patient_id,name,dob,gender,blood_group,height_cm,weight_kg,location,contact,emergency_person,...,disease,patient_status,severity,department_admitted,current_department,bed_type,stage_danger,surgery_required,treatment_protocol,followup_date
0,473b2c48,Tara Brown,1941-07-18,Male,AB+,174,46,"East Timfurt, Nevada, Estonia","(810)490-0925, georgejennifer@example.net",Erica Cobb (Sibling),...,Covid-19,Discharged,Moderate,Radiotherapy,TB Care,ICU,Stage I,Yes,Chemotherapy,2025-01-20
1,cd5e23f0,Adrian Pratt,2017-07-19,Female,O-,167,66,"Delgadoport, Iowa, China","+1-400-420-8049, mary24@example.org",Paige Flores (Parent),...,Cancer,Critical,Critical,TB Care,TB Care,General,Stage II,Under Consideration,Radiotherapy,2025-09-06
2,5e386442,Derrick Duncan,1988-10-23,Male,AB+,172,58,"West Kristopher, Michigan, Singapore","001-203-630-7557x3593, jennifer88@example.com",William Smith (Friend),...,Covid-19,Discharged,Critical,Radiotherapy,Oncology,General,Critical,Yes,Chemotherapy,2025-05-28


In [ ]:
his_d_patients_dataframe['country'].head(5)

0    <NA>
1    <NA>
2    <NA>
3    <NA>
4    <NA>
Name: country, dtype: object

In [ ]:
# print("his_a_patients_dataframe columns : ",his_a_patients_dataframe.columns)
print("\nhis_d_patients_dataframe columns : ",his_d_patients_dataframe.columns)


his_d_patients_dataframe columns :  Index(['HIS', 'patient_id', 'name', 'dob', 'gender', 'blood_group',
       'height_cm', 'weight_kg', 'location', 'contact', 'emergency_person',
       'insurance_id', 'insurance_provider', 'primary_physician',
       'medical_history', 'disease', 'patient_status', 'severity',
       'department_admitted', 'current_department', 'bed_type', 'stage_danger',
       'surgery_required', 'treatment_protocol', 'followup_date', 'phone',
       'last_name', 'email', 'specialty', 'postal_code', 'state', 'time_bound',
       'city', 'emergency_contact_name', 'emergency_contact_relation', 'bp',
       'hb', 'emergency_contact_phone', 'allergies', 'care_instructions',
       'chronic_conditions', 'first_name', 'country', 'duration_days'],
      dtype='object')


In [ ]:
his_d_patients_dataframe[['first_name', 'last_name']] = his_d_patients_dataframe['name'].str.split(' ', n=1, expand=True)

his_d_patients_dataframe[['emergency_contact_name', 'emergency_contact_relation']] = (
    his_d_patients_dataframe['emergency_person']
    .str.extract(r'(?P<emergency_contact_name>.+) \((?P<emergency_contact_relation>.+)\)')
)

his_d_patients_dataframe[['phone', 'email']] = his_d_patients_dataframe['contact'].str.split(',', n=1, expand=True)
his_d_patients_dataframe['phone'] = his_d_patients_dataframe['phone'].str.strip()
his_d_patients_dataframe['email'] = his_d_patients_dataframe['email'].str.strip()

his_d_patients_dataframe[['city', 'state', 'country']] = his_d_patients_dataframe['location'].str.split(',', n=2, expand=True)
his_d_patients_dataframe['city'] = his_d_patients_dataframe['city'].str.strip()
his_d_patients_dataframe['state'] = his_d_patients_dataframe['state'].str.strip()
his_d_patients_dataframe['country'] = his_d_patients_dataframe['country'].str.strip()


In [ ]:
cols_a = set(his_a_patients_dataframe.columns)
cols_c = set(his_c_patients_dataframe.columns)

same_columns = cols_a.intersection(cols_c)

diff_columns_a = cols_a - cols_c
diff_columns_c = cols_c - cols_a

same_columns, diff_columns_a, diff_columns_c


({'HIS',
  'allergies',
  'bed_type',
  'blood_group',
  'bp',
  'care_instructions',
  'chronic_conditions',
  'city',
  'contact',
  'country',
  'current_department',
  'department_admitted',
  'disease',
  'dob',
  'duration_days',
  'email',
  'emergency_contact_name',
  'emergency_contact_phone',
  'emergency_contact_relation',
  'emergency_person',
  'first_name',
  'followup_date',
  'gender',
  'hb',
  'height_cm',
  'insurance_id',
  'insurance_provider',
  'last_name',
  'location',
  'medical_history',
  'patient_id',
  'patient_status',
  'phone',
  'postal_code',
  'primary_physician',
  'severity',
  'specialty',
  'stage_danger',
  'state',
  'surgery_required',
  'time_bound',
  'treatment_protocol',
  'weight_kg'},
 {'name'},
 set())

# Assembing Patients

In [ ]:
all_columns = (
    list(his_a_patients_dataframe.columns) +
    list(his_b_patients_dataframe.columns) +
    list(his_c_patients_dataframe.columns) +
    list(his_d_patients_dataframe.columns)
)

common_columns_dict = {col: all_columns.count(col) for col in set(all_columns)}

print(len(common_columns_dict))

44


In [ ]:
print(common_columns_dict)

{'phone': 4, 'height_cm': 4, 'dob': 4, 'name': 4, 'patient_status': 4, 'last_name': 4, 'email': 4, 'emergency_person': 4, 'specialty': 4, 'postal_code': 4, 'patient_id': 4, 'state': 4, 'contact': 4, 'severity': 4, 'department_admitted': 4, 'time_bound': 4, 'city': 4, 'disease': 4, 'emergency_contact_name': 4, 'bed_type': 4, 'emergency_contact_relation': 4, 'primary_physician': 4, 'treatment_protocol': 4, 'bp': 4, 'blood_group': 4, 'surgery_required': 4, 'hb': 4, 'current_department': 4, 'emergency_contact_phone': 4, 'location': 4, 'insurance_provider': 4, 'stage_danger': 4, 'gender': 4, 'followup_date': 4, 'HIS': 4, 'allergies': 4, 'medical_history': 4, 'care_instructions': 4, 'chronic_conditions': 4, 'insurance_id': 4, 'first_name': 4, 'country': 4, 'weight_kg': 4, 'duration_days': 4}


In [ ]:
for k,v in common_columns_dict.items():
    if v == 4:
        print(v,':',k)

    
for k,v in common_columns_dict.items():
    if v == 3:
        print(v,':',k)
    
for k,v in common_columns_dict.items():
    if v == 2:
        print(v,':',k)

for k,v in common_columns_dict.items():
    if v == 1:
        print(v,':',k)

4 : phone
4 : height_cm
4 : dob
4 : name
4 : patient_status
4 : last_name
4 : email
4 : emergency_person
4 : specialty
4 : postal_code
4 : patient_id
4 : state
4 : contact
4 : severity
4 : department_admitted
4 : time_bound
4 : city
4 : disease
4 : emergency_contact_name
4 : bed_type
4 : emergency_contact_relation
4 : primary_physician
4 : treatment_protocol
4 : bp
4 : blood_group
4 : surgery_required
4 : hb
4 : current_department
4 : emergency_contact_phone
4 : location
4 : insurance_provider
4 : stage_danger
4 : gender
4 : followup_date
4 : HIS
4 : allergies
4 : medical_history
4 : care_instructions
4 : chronic_conditions
4 : insurance_id
4 : first_name
4 : country
4 : weight_kg
4 : duration_days


In [ ]:
his_c_patients_dataframe.rename(columns={
    'contact_phone': 'phone',
    'contact_email': 'email',
    'emergency_person_name': 'emergency_contact_name',
    'emergency_person_relation': 'emergency_contact_relation',
    'emergency_person_phone': 'emergency_contact_phone'
}, inplace=True)

print(his_c_patients_dataframe.columns)

Index(['patient_id', 'name', 'dob', 'gender', 'blood_group', 'height_cm',
       'weight_kg', 'medical_history', 'allergies', 'insurance_id',
       'insurance_provider', 'primary_physician', 'specialty', 'duration_days',
       'care_instructions', 'time_bound', 'phone', 'email',
       'emergency_contact_name', 'emergency_contact_relation',
       'emergency_contact_phone', 'department_admitted', 'current_department',
       'bed_type', 'patient_status', 'severity'],
      dtype='object')


In [ ]:
his_a_patients_dataframe["HIS"] = "HIS_A"
his_b_patients_dataframe["HIS"] = "HIS_B"
his_c_patients_dataframe["HIS"] = "HIS_C"
his_d_patients_dataframe["HIS"] = "HIS_D"

for df_name in ["his_a_patients_dataframe", "his_b_patients_dataframe", "his_c_patients_dataframe", "his_d_patients_dataframe"]:
    df = globals()[df_name]
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index("HIS")))
    globals()[df_name] = df[cols]


In [ ]:
his_a_patients_dataframe.head(2)

,HIS,patient_id,first_name,last_name,dob,gender,blood_group,height_cm,weight_kg,country,...,bp,surgery_required,hb,location,stage_danger,followup_date,allergies,care_instructions,chronic_conditions,duration_days
0,HIS_A,81f91df9,Grace,Andrews,1957-07-28,Male,AB+,191,93,India,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,HIS_A,dfd520ae,Sherri,Smith,1994-04-02,Male,A-,161,98,India,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
his_c_patients_dataframe[['first_name', 'last_name']] = his_c_patients_dataframe['name'].str.split(' ', n=1, expand=True)

his_c_patients_dataframe.drop(columns=['name'],inplace=True)

In [ ]:
print(his_c_patients_dataframe['last_name'])

0           Burns
1          Jordan
2        Richards
3        Williams
4            Wood
          ...    
495    Montgomery
496        Wright
497          Odom
498       Schmidt
499       Edwards
Name: last_name, Length: 500, dtype: object


In [ ]:
for k,v in common_columns_dict.items():
    if v == 4:
        print(v,':',k)

    
for k,v in common_columns_dict.items():
    if v == 3:
        print(v,':',k)
    
for k,v in common_columns_dict.items():
    if v == 2:
        print(v,':',k)

for k,v in common_columns_dict.items():
    if v == 1:
        print(v,':',k)

4 : height_cm
4 : dob
4 : patient_status
4 : patient_id
4 : severity
4 : department_admitted
4 : bed_type
4 : primary_physician
4 : blood_group
4 : current_department
4 : insurance_provider
4 : gender
4 : medical_history
4 : insurance_id
4 : weight_kg
3 : disease
2 : phone
2 : name
2 : last_name
2 : email
2 : postal_code
2 : state
2 : city
2 : emergency_contact_name
2 : emergency_contact_relation
2 : emergency_contact_phone
2 : allergies
2 : first_name
2 : country
1 : emergency_person
1 : specialty
1 : contact
1 : emergency_person_relation
1 : time_bound
1 : emergency_person_name
1 : treatment_protocol
1 : bp
1 : contact_email
1 : surgery_required
1 : hb
1 : location
1 : stage_danger
1 : emergency_person_phone
1 : followup_date
1 : care_instructions
1 : chronic_conditions
1 : contact_phone
1 : duration_days


In [ ]:
import pandas as pd

all_columns = set()
for df in [his_a_patients_dataframe, his_b_patients_dataframe, his_c_patients_dataframe, his_d_patients_dataframe]:
    all_columns.update(df.columns)

for df in [his_a_patients_dataframe, his_b_patients_dataframe, his_c_patients_dataframe, his_d_patients_dataframe]:
    for col in all_columns:
        if col not in df.columns:
            df[col] = pd.NA
    df = df[list(all_columns)]  
    
assembled_patients_dataframe = pd.concat(
    [his_a_patients_dataframe, his_b_patients_dataframe, his_c_patients_dataframe, his_d_patients_dataframe],
    ignore_index=True
)


C:\Users\abhis\AppData\Local\Temp\ipykernel_18576\3449595937.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  assembled_patients_dataframe = pd.concat(


In [ ]:
assembled_patients_dataframe[:]

,HIS,patient_id,first_name,last_name,dob,gender,blood_group,height_cm,weight_kg,country,...,bp,surgery_required,hb,location,stage_danger,followup_date,allergies,care_instructions,chronic_conditions,duration_days
0,HIS_A,81f91df9,Grace,Andrews,1957-07-28,Male,AB+,191,93,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,HIS_A,dfd520ae,Sherri,Smith,1994-04-02,Male,A-,161,98,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,HIS_A,f6173246,Dana,Phillips,1941-03-10,Female,A-,159,56,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,HIS_A,79326454,Paul,Campbell,1941-07-01,Male,AB-,196,110,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,HIS_A,59978f9f,Daniel,Jones,2016-03-14,Male,AB-,192,120,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,HIS_D,9a517f68,NaN,NaN,1953-03-08,Female,AB-,192,112,NaN,...,NaN,Yes,NaN,"Patriciaside, California, Guadeloupe",Stage III,2024-09-19,NaN,NaN,NaN,<NA>
2496,HIS_D,84ebfef0,NaN,NaN,1971-04-11,Male,O+,198,49,NaN,...,NaN,Under Consideration,NaN,"East Eric, Alaska, Indonesia",Critical,2025-04-04,NaN,NaN,NaN,<NA>
2497,HIS_D,d78e53cc,NaN,NaN,1979-02-23,Female,O-,172,63,NaN,...,NaN,Yes,NaN,"Cynthiafort, Washington, Hong Kong",Stage III,2024-12-17,NaN,NaN,NaN,<NA>
2498,HIS_D,75230abe,NaN,NaN,1989-03-23,Female,B+,175,103,NaN,...,NaN,No,NaN,"South Crystalbury, Utah, Antigua and Barbuda",Stage IV,2024-12-08,NaN,NaN,NaN,<NA>


In [ ]:
assembled_patients_dataframe.isna().sum()

patient_id                       0
first_name                     900
last_name                      900
dob                              0
gender                           0
blood_group                      0
height_cm                      400
weight_kg                      400
country                        900
state                          900
city                           900
postal_code                    900
phone                          400
email                          400
emergency_contact_name         400
emergency_contact_relation     400
emergency_contact_phone        400
insurance_id                     0
insurance_provider               0
primary_physician                0
medical_history                125
disease                        500
patient_status                   0
severity                         0
department_admitted              0
current_department               0
bed_type                         0
HIS                              0
name                

In [ ]:
assembled_patients_dataframe.sort_values(by="HIS")

,HIS,patient_id,first_name,last_name,dob,gender,blood_group,height_cm,weight_kg,country,...,bp,surgery_required,hb,location,stage_danger,followup_date,allergies,care_instructions,chronic_conditions,duration_days
0,HIS_A,81f91df9,Grace,Andrews,1957-07-28,Male,AB+,191,93,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
804,HIS_A,3fff747c,Christopher,Anderson,1997-11-10,Male,A+,190,59,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
803,HIS_A,fc47c028,Richard,Smith,1976-03-18,Female,O+,195,89,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
802,HIS_A,ef8b5005,Kaitlyn,Duran,1938-06-25,Male,O+,167,78,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
801,HIS_A,8e70babd,Pamela,Sanders,1995-11-21,Male,A-,188,68,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,HIS_D,bb009e5e,NaN,NaN,2019-06-25,Male,AB+,168,116,NaN,...,NaN,Under Consideration,NaN,"New Jason, South Dakota, Bulgaria",Stage II,2024-11-11,NaN,NaN,NaN,<NA>
2226,HIS_D,cfe435e7,NaN,NaN,1943-08-01,Male,A-,157,74,NaN,...,NaN,No,NaN,"Briannachester, Illinois, Chad",Critical,2025-09-15,NaN,NaN,NaN,<NA>
2225,HIS_D,d189fcc6,NaN,NaN,1977-07-07,Male,AB+,150,114,NaN,...,NaN,No,NaN,"Port Stephenview, Montana, Malawi",Stage III,2026-09-23,NaN,NaN,NaN,<NA>
2235,HIS_D,e0317274,NaN,NaN,1964-03-07,Female,A-,168,57,NaN,...,NaN,Under Consideration,NaN,"East Hayley, Maine, Libyan Arab Jamahiriya",Stage II,2024-10-08,NaN,NaN,NaN,<NA>


In [ ]:
assembled_patients_dataframe.to_csv("assembled_patients_dataframe1.csv", index=False)


In [ ]:
assembled_patients_dataframe.drop(columns=['name', 'emergency_person', 'contact', 'location'], inplace=True)


So this is how it looks !!

In [ ]:
print(assembled_patients_dataframe.shape)

(2500, 40)


In [ ]:
assembled_patients_dataframe.head(5)

,HIS,patient_id,first_name,last_name,dob,gender,blood_group,height_cm,weight_kg,country,...,treatment_protocol,bp,surgery_required,hb,stage_danger,followup_date,allergies,care_instructions,chronic_conditions,duration_days
0,HIS_A,81f91df9,Grace,Andrews,1957-07-28,Male,AB+,191,93,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,HIS_A,dfd520ae,Sherri,Smith,1994-04-02,Male,A-,161,98,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,HIS_A,f6173246,Dana,Phillips,1941-03-10,Female,A-,159,56,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,HIS_A,79326454,Paul,Campbell,1941-07-01,Male,AB-,196,110,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,HIS_A,59978f9f,Daniel,Jones,2016-03-14,Male,AB-,192,120,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [ ]:
import os
import sqlite3

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

unified_dir = os.path.join(project_root, "UNIFIED_DATA")
os.makedirs(unified_dir, exist_ok=True)

db_path = os.path.join(unified_dir, "unified_patients.db")

conn = sqlite3.connect(db_path)

assembled_patients_dataframe.to_sql("patients", conn, if_exists="replace", index=False)

conn.close()

print("Unified patient data saved, ready to upload !")

Unified patient data saved, ready to upload !


In [ ]:
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

unified_dir = os.path.join(project_root, "UNIFIED_DATA")
os.makedirs(unified_dir, exist_ok=True)

db_path = os.path.join(unified_dir, "unified_patients.csv")

assembled_patients_dataframe.to_csv(db_path)


print("Unified patient data saved, ready to upload !")

Unified patient data saved, ready to upload !


# Patients database assembled


# Now to work on Staff data

# HIS_A Staff data

In [ ]:
import requests
import sqlite3
import pandas as pd
import tempfile

dropbox_url = "https://www.dropbox.com/scl/fi/yhsbboc37dvwztxby4304/staff.db?rlkey=q88ft8h7riqnd9kpwlybdvm1u&st=8ybp49wp&dl=1"

with tempfile.NamedTemporaryFile(delete=False, suffix=".db") as tmp_db:
    response = requests.get(dropbox_url)
    response.raise_for_status()  
    tmp_db.write(response.content)
    tmp_db_path = tmp_db.name

conn = sqlite3.connect(tmp_db_path)


table_names = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", table_names)

his_a_staff_dataframe = pd.read_sql("SELECT * FROM staff", conn)

print(his_a_staff_dataframe.head())

conn.close()


Tables in DB:     name
0  staff
   staff_id first_name last_name         dob  gender   department  \
0  Sf48c6c0      Tammy  Peterson  1989-01-15    Male    Radiology   
1  S5d7bb3d    Michael     Jones  1982-12-01  Female  Orthopedics   
2  Sf0cd7cc     Dennis    Garcia  1983-01-04    Male   Cardiology   
3  S2bf065f      Kelly    Miller  1975-01-14    Male  Orthopedics   
4  S3fcf0df       Mark   Hampton  1990-12-19    Male   Cardiology   

    designation joining_date  salary          phone  ...  \
0      Resident   2008-05-02  118791  3594460015773  ...   
1  Senior Nurse   2010-10-28   92715  3876837507347  ...   
2        Doctor   2025-02-18   65953  7977296260718  ...   
3    Technician   2017-10-18   66934  4041648664033  ...   
4         Admin   2009-10-14   97541  0474769328292  ...   

                    address_line1 address_line2    city        state country  \
0  13207 Johnson Courts Suite 238      Apt. 092  Mumbai  Maharashtra   India   
1     66045 Ashley Alley Apt. 43

In [ ]:
his_a_staff_dataframe.columns

Index(['staff_id', 'first_name', 'last_name', 'dob', 'gender', 'department',
       'designation', 'joining_date', 'salary', 'phone', 'email',
       'address_line1', 'address_line2', 'city', 'state', 'country',
       'postal_code', 'emergency_contact_name', 'emergency_contact_relation',
       'emergency_contact_phone', 'work_shift'],
      dtype='object')

In [ ]:
his_a_staff_dataframe.head(3)

,staff_id,first_name,last_name,dob,gender,department,designation,joining_date,salary,phone,...,address_line1,address_line2,city,state,country,postal_code,emergency_contact_name,emergency_contact_relation,emergency_contact_phone,work_shift
0,Sf48c6c0,Tammy,Peterson,1989-01-15,Male,Radiology,Resident,2008-05-02,118791,3594460015773,...,13207 Johnson Courts Suite 238,Apt. 092,Mumbai,Maharashtra,India,52619,Dr. Jennifer West,Spouse,2779608969165,Evening
1,S5d7bb3d,Michael,Jones,1982-12-01,Female,Orthopedics,Senior Nurse,2010-10-28,92715,3876837507347,...,66045 Ashley Alley Apt. 435,Suite 677,Pune,Maharashtra,India,19967,Stacey Morrow,Sibling,7699128708747,Morning
2,Sf0cd7cc,Dennis,Garcia,1983-01-04,Male,Cardiology,Doctor,2025-02-18,65953,7977296260718,...,882 Jackie Burg,Apt. 844,Pune,Maharashtra,India,12240,Michael Contreras,Spouse,2862947961560,Morning


In [132]:
his_a_staff_dataframe['address_line1'] = his_a_staff_dataframe['address_line1'].astype(str) + ', ' + his_a_staff_dataframe['address_line2'].astype(str)
his_a_staff_dataframe.drop(columns=['address_line2'], inplace=True)

# HIS_B Staff data

In [ ]:
import requests
import json

dropbox_url_his_b = "https://www.dropbox.com/scl/fi/fp7wnf1tqoo8xld8fg6y6/staff.json?rlkey=qreffrgl2ofjb6z8h1zkh66cx&st=rysgn7sz&dl=1"

response = requests.get(dropbox_url_his_b)
response.raise_for_status() 

his_b_staff_json = response.json()


In [ ]:
pprint(his_b_staff_json[:3])

[{'address': {'city': 'Pune',
              'country': 'India',
              'line1': '85833 Kevin Views',
              'line2': 'Apt. 167',
              'postal_code': '37405',
              'state': 'Maharashtra'},
  'contact': {'email': 'robinsonsamuel@hamilton-wong.com',
              'phone': '8017040646801'},
  'department': 'Pediatrics',
  'designation': 'Doctor',
  'dob': '1967-04-27',
  'emergency_person': {'name': 'Sarah Hernandez',
                       'phone': '5791222666934',
                       'relation': 'Sibling'},
  'gender': 'Male',
  'joining_date': '2006-02-19',
  'name': {'first': 'Pamela', 'last': 'Griffith'},
  'salary': 77629,
  'staff_id': 'Scc60240',
  'work_shift': 'Evening'},
 {'address': {'city': 'Nagpur',
              'country': 'India',
              'line1': '584 Johnson Station',
              'line2': 'Apt. 514',
              'postal_code': '58506',
              'state': 'Maharashtra'},
  'contact': {'email': 'christinagallagher@estrada-cab

In [ ]:
import pandas as pd

his_b_staff_flat = []
for staff in his_b_staff_json:
    flat = {}

    flat['staff_id'] = staff.get('staff_id')
    flat['first_name'] = staff.get('name', {}).get('first')
    flat['last_name'] = staff.get('name', {}).get('last')
    flat['dob'] = staff.get('dob')
    flat['gender'] = staff.get('gender')
    flat['department'] = staff.get('department')
    flat['designation'] = staff.get('designation')
    flat['joining_date'] = staff.get('joining_date')
    flat['salary'] = staff.get('salary')
    flat['work_shift'] = staff.get('work_shift')
    
    flat['phone'] = staff.get('contact', {}).get('phone')
    flat['email'] = staff.get('contact', {}).get('email')
    
    flat['address_line1'] = staff.get('address', {}).get('line1')
    flat['address_line2'] = staff.get('address', {}).get('line2')
    flat['city'] = staff.get('address', {}).get('city')
    flat['state'] = staff.get('address', {}).get('state')
    flat['country'] = staff.get('address', {}).get('country')
    flat['postal_code'] = staff.get('address', {}).get('postal_code')
    
    flat['emergency_contact_name'] = staff.get('emergency_person', {}).get('name')
    flat['emergency_contact_relation'] = staff.get('emergency_person', {}).get('relation')
    flat['emergency_contact_phone'] = staff.get('emergency_person', {}).get('phone')
    
    
    his_b_staff_flat.append(flat)

his_b_staff_dataframe = pd.DataFrame(his_b_staff_flat)

print(his_b_staff_dataframe.columns)


Index(['staff_id', 'first_name', 'last_name', 'dob', 'gender', 'department',
       'designation', 'joining_date', 'salary', 'work_shift', 'phone', 'email',
       'address_line1', 'address_line2', 'city', 'state', 'country',
       'postal_code', 'emergency_contact_name', 'emergency_contact_relation',
       'emergency_contact_phone'],
      dtype='object')


In [ ]:
his_b_staff_dataframe.head(3)

,staff_id,first_name,last_name,dob,gender,department,designation,joining_date,salary,work_shift,...,email,address_line1,address_line2,city,state,country,postal_code,emergency_contact_name,emergency_contact_relation,emergency_contact_phone
0,Scc60240,Pamela,Griffith,1967-04-27,Male,Pediatrics,Doctor,2006-02-19,77629,Evening,...,robinsonsamuel@hamilton-wong.com,85833 Kevin Views,Apt. 167,Pune,Maharashtra,India,37405,Sarah Hernandez,Sibling,5791222666934
1,S76e902e,Emily,Stevens,1990-09-20,Male,Orthopedics,Doctor,2013-03-25,95612,Morning,...,christinagallagher@estrada-cabrera.com,584 Johnson Station,Apt. 514,Nagpur,Maharashtra,India,58506,Amy Porter,Parent,1453425857341
2,Sac91a4c,Sandra,Diaz,1981-04-22,Male,Orthopedics,Doctor,2012-05-14,56434,Night,...,lgriffith@green.com,03635 Holloway Springs,Suite 155,Nashik,Maharashtra,India,95360,Michele Hickman,Friend,1778181545122


In [ ]:
print(his_a_staff_dataframe.shape[1] == his_b_staff_dataframe.shape[1])

True


# HIS_C Staff

In [ ]:

import pandas as pd
import requests
from io import StringIO

his_c_csv_url = "https://www.dropbox.com/scl/fi/55g8tpsvlwy9w26hn4w5v/staff.csv?rlkey=lgld0kaf2h4nlpglnvouwzzit&st=uddy0fid&dl=1"

response = requests.get(his_c_csv_url)
response.raise_for_status()

his_c_staff_dataframe = pd.read_csv(StringIO(response.text))

print(his_c_staff_dataframe.shape, his_c_staff_dataframe.columns)


(50, 14) Index(['staff_id', 'name', 'role', 'specialty_assigned', 'joining_date',
       'experience_years', 'qualification', 'shift', 'contact_phone',
       'contact_email', 'address', 'emergency_person_name',
       'emergency_person_relation', 'emergency_person_phone'],
      dtype='object')


In [ ]:
his_c_staff_dataframe.head(3)

,staff_id,name,role,specialty_assigned,joining_date,experience_years,qualification,shift,contact_phone,contact_email,address,emergency_person_name,emergency_person_relation,emergency_person_phone
0,b34909d4,Ryan Bennett,Assistant,Dermatology – Skincare,2013-07-04,26,MBBS,Night,8649345827485,sellersashlee@example.com,"166 Briggs Run\nNew Matthewmouth, VI 50870",Zachary Jones,Parent,8869242970992
1,ce1db19f,Tony Griffin,Assistant,Trichology – Hair Care,2002-12-02,10,MBBS,Morning,2930276080411,dawnoliver@example.net,"80919 Burgess Junctions\nNew Donna, ME 41001",Tony Hoffman,Spouse,6590436633648
2,4debdafd,Nicholas Rice,Therapist,Ophthalmology – Eye Care,2016-05-06,17,Nursing,Evening,4168165043859,robert62@example.org,6783 Zachary Greens Apt. 368\nNorth Josephhave...,Christine Martin,Friend,8326788084513


In [ ]:
print(his_a_staff_dataframe.columns, '\n',his_c_staff_dataframe.columns)

Index(['staff_id', 'first_name', 'last_name', 'dob', 'gender', 'department',
       'designation', 'joining_date', 'salary', 'phone', 'email',
       'address_line1', 'address_line2', 'city', 'state', 'country',
       'postal_code', 'emergency_contact_name', 'emergency_contact_relation',
       'emergency_contact_phone', 'work_shift'],
      dtype='object') 
 Index(['staff_id', 'name', 'role', 'specialty_assigned', 'joining_date',
       'experience_years', 'qualification', 'shift', 'contact_phone',
       'contact_email', 'address', 'emergency_person_name',
       'emergency_person_relation', 'emergency_person_phone'],
      dtype='object')


In [121]:
print(his_a_staff_dataframe.head(5), '\n\n\n',his_c_staff_dataframe.head(5))

   staff_id first_name last_name         dob  gender   department  \
0  Sf48c6c0      Tammy  Peterson  1989-01-15    Male    Radiology   
1  S5d7bb3d    Michael     Jones  1982-12-01  Female  Orthopedics   
2  Sf0cd7cc     Dennis    Garcia  1983-01-04    Male   Cardiology   
3  S2bf065f      Kelly    Miller  1975-01-14    Male  Orthopedics   
4  S3fcf0df       Mark   Hampton  1990-12-19    Male   Cardiology   

    designation joining_date  salary          phone  ...  \
0      Resident   2008-05-02  118791  3594460015773  ...   
1  Senior Nurse   2010-10-28   92715  3876837507347  ...   
2        Doctor   2025-02-18   65953  7977296260718  ...   
3    Technician   2017-10-18   66934  4041648664033  ...   
4         Admin   2009-10-14   97541  0474769328292  ...   

                    address_line1 address_line2    city        state country  \
0  13207 Johnson Courts Suite 238      Apt. 092  Mumbai  Maharashtra   India   
1     66045 Ashley Alley Apt. 435     Suite 677    Pune  Maharas

In [125]:
his_c_staff_dataframe['address'].head(10)

0           166 Briggs Run\nNew Matthewmouth, VI 50870
1         80919 Burgess Junctions\nNew Donna, ME 41001
2    6783 Zachary Greens Apt. 368\nNorth Josephhave...
3           21136 Johnson Harbor\nEast Kayla, DE 05665
4    9690 Jennifer Coves Apt. 997\nEast Richard, WA...
5                     Unit 4392 Box 5729\nDPO AE 85867
6               2081 Russell Keys\nWest John, PA 86740
7    7227 Allison Ferry Apt. 724\nNathantown, WA 94028
8                   091 Yu Brook\nThomasland, AR 60546
9             8843 Cynthia Mills\nWilsonfort, DE 91385
Name: address, dtype: object

In [129]:
his_c_staff_dataframe['address'][2]

'6783 Zachary Greens Apt. 368\nNorth Josephhaven, TN 97970'

In [ ]:
his_c_staff_dataframe[['first_name', 'last_name']] = his_c_staff_dataframe['name'].str.split(' ', n=1, expand=True)

address_split = his_c_staff_dataframe['address'].str.split('\n', expand=True)
his_c_staff_dataframe['address_line1'] = address_split[0]

city_state_postal = address_split[1].str.extract(r'(?P<city>.*), (?P<state>\w+) (?P<postal_code>.*)')
his_c_staff_dataframe['city'] = city_state_postal['city']
his_c_staff_dataframe['state'] = city_state_postal['state']
his_c_staff_dataframe['postal_code'] = city_state_postal['postal_code']

his_c_staff_dataframe.rename(columns={
    'role': 'designation',
    'specialty_assigned': 'department',
    'contact_phone': 'phone',
    'contact_email': 'email',
    'emergency_person_name': 'emergency_contact_name',
    'emergency_person_relation': 'emergency_contact_relation',
    'emergency_person_phone': 'emergency_contact_phone',
    'shift': 'work_shift'
}, inplace=True)

his_c_staff_dataframe.drop(columns=['name', 'address'], inplace=True)


In [131]:
print(his_a_staff_dataframe.head(5), '\n\n\n',his_c_staff_dataframe.head(5))

   staff_id first_name last_name         dob  gender   department  \
0  Sf48c6c0      Tammy  Peterson  1989-01-15    Male    Radiology   
1  S5d7bb3d    Michael     Jones  1982-12-01  Female  Orthopedics   
2  Sf0cd7cc     Dennis    Garcia  1983-01-04    Male   Cardiology   
3  S2bf065f      Kelly    Miller  1975-01-14    Male  Orthopedics   
4  S3fcf0df       Mark   Hampton  1990-12-19    Male   Cardiology   

    designation joining_date  salary          phone  ...  \
0      Resident   2008-05-02  118791  3594460015773  ...   
1  Senior Nurse   2010-10-28   92715  3876837507347  ...   
2        Doctor   2025-02-18   65953  7977296260718  ...   
3    Technician   2017-10-18   66934  4041648664033  ...   
4         Admin   2009-10-14   97541  0474769328292  ...   

                    address_line1 address_line2    city        state country  \
0  13207 Johnson Courts Suite 238      Apt. 092  Mumbai  Maharashtra   India   
1     66045 Ashley Alley Apt. 435     Suite 677    Pune  Maharas

# HIS_D Staff

In [ ]:
import requests
import sqlite3
import pandas as pd
import tempfile

dropbox_url = "https://www.dropbox.com/scl/fi/sxnemu2qw1bsae5hqw39f/staff.db?rlkey=l0ryopq04ue21o0egkwupldi8&st=7cypcki4&dl=1"

with tempfile.NamedTemporaryFile(delete=False, suffix=".db") as tmp_db:
    response = requests.get(dropbox_url)
    response.raise_for_status() 
    tmp_db.write(response.content)
    tmp_db_path = tmp_db.name

conn = sqlite3.connect(tmp_db_path)

table_names = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", table_names)

his_d_staff_dataframe = pd.read_sql("SELECT * FROM staff", conn)

conn.close()

Tables in DB:     name
0  staff


In [ ]:
his_d_staff_dataframe.head(3)

,staff_id,name,dob,gender,position,department,contact,salary,date_joined,specialization,experience_years,certifications,shift,email,address,emergency_contact,emergency_relation
0,0d335115,Daniel Thompson,1999-03-16,Female,Admin,Neurosurgery,"734.206.4478x8621, natalie43@example.org",62994.32,2016-01-24,TB,40,"Environmental manager, Sims Inc",Rotational,fferrell@example.net,"Randymouth, North Dakota, New Caledonia",Nicholas Myers,Offspring
1,ed44447f,Jason Pena,1975-08-03,Female,Admin,ICU,"923-561-8992x47238, ckramer@example.com",86991.49,2018-05-16,Radiotherapy,25,"Banker, Baker-Perez",Night,acunningham@example.org,"West Michael, Rhode Island, Ukraine",Jason Ball,Friend
2,29bb1bb4,Mikayla Ramirez,1998-05-01,Female,Nurse,Neurosurgery,"(404)271-7693, jeremy10@example.com",128080.34,2019-10-24,HIV,30,"Electronics engineer, Lara PLC",Rotational,durankayla@example.org,"North John, Louisiana, Portugal",Tamara Thompson,Friend


In [ ]:
print(his_a_staff_dataframe.columns, '\n',his_d_staff_dataframe.columns)

Index(['staff_id', 'first_name', 'last_name', 'dob', 'gender', 'department',
       'designation', 'joining_date', 'salary', 'phone', 'email',
       'address_line1', 'address_line2', 'city', 'state', 'country',
       'postal_code', 'emergency_contact_name', 'emergency_contact_relation',
       'emergency_contact_phone', 'work_shift'],
      dtype='object') 
 Index(['staff_id', 'name', 'dob', 'gender', 'position', 'department',
       'contact', 'salary', 'date_joined', 'specialization',
       'experience_years', 'certifications', 'shift', 'email', 'address',
       'emergency_contact', 'emergency_relation'],
      dtype='object')


In [135]:
print('HIS_A\n',his_a_staff_dataframe.head(5), '\n\n\nHIS_C\n',his_c_staff_dataframe.head(5), '\n\n\nHIS_D\n',his_d_staff_dataframe.head(5))

HIS_A
    staff_id first_name last_name         dob  gender   department  \
0  Sf48c6c0      Tammy  Peterson  1989-01-15    Male    Radiology   
1  S5d7bb3d    Michael     Jones  1982-12-01  Female  Orthopedics   
2  Sf0cd7cc     Dennis    Garcia  1983-01-04    Male   Cardiology   
3  S2bf065f      Kelly    Miller  1975-01-14    Male  Orthopedics   
4  S3fcf0df       Mark   Hampton  1990-12-19    Male   Cardiology   

    designation joining_date  salary          phone  \
0      Resident   2008-05-02  118791  3594460015773   
1  Senior Nurse   2010-10-28   92715  3876837507347   
2        Doctor   2025-02-18   65953  7977296260718   
3    Technician   2017-10-18   66934  4041648664033   
4         Admin   2009-10-14   97541  0474769328292   

                               email  \
0          roybarker@patel-ross.info   
1  martineznicole@steele-hawkins.com   
2           kbond@stewart-castro.com   
3                  julie31@johns.net   
4          bakermargaret@aguilar.com   

      

In [ ]:
from faker import Faker
fake = Faker()

his_d_staff_dataframe[['first_name', 'last_name']] = his_d_staff_dataframe['name'].str.split(' ', n=1, expand=True)

his_d_staff_dataframe.rename(columns={
    'position': 'designation',
    'shift': 'work_shift',
    'emergency_contact': 'emergency_contact_name',
    'emergency_relation': 'emergency_contact_relation'
}, inplace=True)

his_d_staff_dataframe['contact'] = [fake.msisdn() for _ in range(len(his_d_staff_dataframe))]
his_d_staff_dataframe.rename(columns={'contact': 'phone'}, inplace=True)

his_d_staff_dataframe[['city', 'state', 'country']] = his_d_staff_dataframe['address'].str.split(',', n=2, expand=True)


In [137]:
print('HIS_A\n',his_a_staff_dataframe.head(5), '\n\n\nHIS_C\n',his_c_staff_dataframe.head(5), '\n\n\nHIS_D\n',his_d_staff_dataframe.head(5))

HIS_A
    staff_id first_name last_name         dob  gender   department  \
0  Sf48c6c0      Tammy  Peterson  1989-01-15    Male    Radiology   
1  S5d7bb3d    Michael     Jones  1982-12-01  Female  Orthopedics   
2  Sf0cd7cc     Dennis    Garcia  1983-01-04    Male   Cardiology   
3  S2bf065f      Kelly    Miller  1975-01-14    Male  Orthopedics   
4  S3fcf0df       Mark   Hampton  1990-12-19    Male   Cardiology   

    designation joining_date  salary          phone  \
0      Resident   2008-05-02  118791  3594460015773   
1  Senior Nurse   2010-10-28   92715  3876837507347   
2        Doctor   2025-02-18   65953  7977296260718   
3    Technician   2017-10-18   66934  4041648664033   
4         Admin   2009-10-14   97541  0474769328292   

                               email  \
0          roybarker@patel-ross.info   
1  martineznicole@steele-hawkins.com   
2           kbond@stewart-castro.com   
3                  julie31@johns.net   
4          bakermargaret@aguilar.com   

      

# Let's assemble HIS's Staff data

In [138]:
his_a_staff_dataframe['HIS'] = 'HIS_A'
his_b_staff_dataframe['HIS'] = 'HIS_B'
his_c_staff_dataframe['HIS'] = 'HIS_C'
his_d_staff_dataframe['HIS'] = 'HIS_D'


In [ ]:
staff_dfs = {
    "HIS_A": his_a_staff_dataframe,
    "HIS_B": his_b_staff_dataframe,
    "HIS_C": his_c_staff_dataframe,
    "HIS_D": his_d_staff_dataframe
}

for his_name, df in staff_dfs.items():
    if 'HIS' in df.columns:
        df.drop(columns=['HIS'], inplace=True)
    
    df.insert(0, 'HIS', his_name)


In [142]:
for data in [his_a_staff_dataframe, his_b_staff_dataframe, his_c_staff_dataframe, his_d_staff_dataframe]:
    print(data.iloc[0],'\n\n\n')

HIS                                                              HIS_A
staff_id                                                      Sf48c6c0
first_name                                                       Tammy
last_name                                                     Peterson
dob                                                         1989-01-15
gender                                                            Male
department                                                   Radiology
designation                                                   Resident
joining_date                                                2008-05-02
salary                                                          118791
phone                                                    3594460015773
email                                        roybarker@patel-ross.info
address_line1                 13207 Johnson Courts Suite 238, Apt. 092
city                                                            Mumbai
state 

In [ ]:
import pandas as pd

staff_dfs = [his_a_staff_dataframe, his_b_staff_dataframe, his_c_staff_dataframe, his_d_staff_dataframe]

assembled_staff_dataframe = pd.concat(staff_dfs, ignore_index=True, sort=False)

assembled_staff_dataframe.head(3)


,HIS,staff_id,first_name,last_name,dob,gender,department,designation,joining_date,salary,...,emergency_contact_phone,work_shift,address_line2,experience_years,qualification,name,date_joined,specialization,certifications,address
0,HIS_A,Sf48c6c0,Tammy,Peterson,1989-01-15,Male,Radiology,Resident,2008-05-02,118791.0,...,2779608969165,Evening,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HIS_A,S5d7bb3d,Michael,Jones,1982-12-01,Female,Orthopedics,Senior Nurse,2010-10-28,92715.0,...,7699128708747,Morning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HIS_A,Sf0cd7cc,Dennis,Garcia,1983-01-04,Male,Cardiology,Doctor,2025-02-18,65953.0,...,2862947961560,Morning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
assembled_staff_dataframe.to_csv('assembled_staff_data.csv')

In [146]:
assembled_staff_dataframe.drop(columns=['address_line2', 'name', 'address'], inplace=True)


In [147]:
assembled_staff_dataframe.to_csv('assembled_staff_data1.csv')

In [ ]:
import sqlite3
import os

os.makedirs('../UNIFIED_DATA', exist_ok=True)

db_path = '../UNIFIED_DATA/unified_staff.db'

conn = sqlite3.connect(db_path)

assembled_staff_dataframe.to_sql('staff', conn, if_exists='replace', index=False)

conn.close()

print('Assembled Staff Data saved in UNIFIED_DATA directory !!')

Assembled Staff Data saved in UNIFIED_DATA directory !!


# Now it's time for Finance data preparation

# HIS_A finance

In [ ]:
import requests
import sqlite3
import pandas as pd
import tempfile

dropbox_url = "https://www.dropbox.com/scl/fi/cbrcpka01uclj7b5mbf9j/finance.db?rlkey=gf2k72ja5phm8d84e853me1yq&st=yrfdlauc&dl=1"

with tempfile.NamedTemporaryFile(delete=False, suffix=".db") as tmp_db:
    response = requests.get(dropbox_url)
    response.raise_for_status()  # ensure we fail if download fails
    tmp_db.write(response.content)
    tmp_db_path = tmp_db.name

conn = sqlite3.connect(tmp_db_path)

table_names = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", table_names)

his_a_finance_dataframe = pd.read_sql("SELECT * FROM finance", conn)

print(his_a_finance_dataframe.columns)

conn.close()

Tables in DB:       name
0  finance
Index(['transaction_id', 'patient_id', 'billing_date', 'amount', 'currency',
       'payment_method', 'insurance_claimed', 'insurance_id',
       'insurance_provider', 'service_code', 'service_description',
       'department', 'staff_id', 'discount_applied', 'net_amount', 'tax',
       'total_amount', 'payment_status', 'due_date', 'receipt_number'],
      dtype='object')


In [150]:
his_a_finance_dataframe.head(3)

,transaction_id,patient_id,billing_date,amount,currency,payment_method,insurance_claimed,insurance_id,insurance_provider,service_code,service_description,department,staff_id,discount_applied,net_amount,tax,total_amount,payment_status,due_date,receipt_number
0,Te7aa90d,2a4cf4e4,2023-07-22,18537.49,INR,UPI,No,INS92486,ABC Insurance,SRV104,Surgery,Emergency,Sa6b9ca3,217.02,18320.47,1648.84,19969.31,Paid,2026-12-31,RCPT309436
1,T7ed7c43,37c9f9de,2025-03-13,18304.87,INR,Cash,No,INS62636,CarePlus,SRV101,MRI Scan,Emergency,Sa6abd83,2588.48,15716.39,1414.48,17130.87,Pending,2024-10-15,RCPT721169
2,Tf4db4c9,4641840c,2024-03-27,4236.74,INR,Debit Card,Yes,INS64416,CarePlus,SRV102,X-Ray,Orthopedics,Scf07bb8,597.63,3639.11,327.52,3966.63,Unpaid,2026-02-25,RCPT452362


# HIS_B finance

In [ ]:
import requests
import json

dropbox_url_his_b = "https://www.dropbox.com/scl/fi/drbu4nik0awf7hh3453f7/finance.json?rlkey=wm21rq76fulooqb9b15ly3ly7&st=suw49lbw&dl=1"

response = requests.get(dropbox_url_his_b)
response.raise_for_status()  
his_b_finance_json = response.json()

In [152]:
his_b_finance_json[:3]

[{'transaction_id': 'T5381df6',
  'patient_id': 'cc594dd8',
  'billing_date': '2025-01-17',
  'amount': 497.95,
  'currency': 'INR',
  'payment_method': 'UPI',
  'insurance_claimed': 'No',
  'insurance_id': 'INS68076',
  'insurance_provider': 'ABC Insurance',
  'service_code': 'SRV102',
  'service_description': 'X-Ray',
  'department': 'Orthopedics',
  'staff_id': 'Sf3ceaaf',
  'discount_applied': 41.73,
  'net_amount': 456.22,
  'tax': 41.06,
  'total_amount': 497.28,
  'payment_status': 'Pending',
  'due_date': '2024-12-12',
  'receipt_number': 'RCPT946873'},
 {'transaction_id': 'T07c3fa5',
  'patient_id': '019a9697',
  'billing_date': '2022-05-20',
  'amount': 5725.15,
  'currency': 'INR',
  'payment_method': 'Cash',
  'insurance_claimed': 'No',
  'insurance_id': 'INS59036',
  'insurance_provider': 'CarePlus',
  'service_code': 'SRV103',
  'service_description': 'Blood Test',
  'department': 'Emergency',
  'staff_id': 'Sa4cc9f4',
  'discount_applied': 705.45,
  'net_amount': 5019.7,

In [153]:
his_b_finance_dataframe = pd.DataFrame(his_b_finance_json)

print(his_b_finance_dataframe.columns)
print(his_b_finance_dataframe.shape)

Index(['transaction_id', 'patient_id', 'billing_date', 'amount', 'currency',
       'payment_method', 'insurance_claimed', 'insurance_id',
       'insurance_provider', 'service_code', 'service_description',
       'department', 'staff_id', 'discount_applied', 'net_amount', 'tax',
       'total_amount', 'payment_status', 'due_date', 'receipt_number'],
      dtype='object')
(120, 20)


In [157]:
print(his_b_finance_dataframe.shape[1] == his_a_finance_dataframe.shape[1])

True


In [158]:
his_b_finance_dataframe.head(3)

,transaction_id,patient_id,billing_date,amount,currency,payment_method,insurance_claimed,insurance_id,insurance_provider,service_code,service_description,department,staff_id,discount_applied,net_amount,tax,total_amount,payment_status,due_date,receipt_number
0,T5381df6,cc594dd8,2025-01-17,497.95,INR,UPI,No,INS68076,ABC Insurance,SRV102,X-Ray,Orthopedics,Sf3ceaaf,41.73,456.22,41.06,497.28,Pending,2024-12-12,RCPT946873
1,T07c3fa5,019a9697,2022-05-20,5725.15,INR,Cash,No,INS59036,CarePlus,SRV103,Blood Test,Emergency,Sa4cc9f4,705.45,5019.70,451.77,5471.47,Unpaid,2026-01-18,RCPT194849
2,T70a560c,49fb997a,2023-03-07,13106.95,INR,Credit Card,Yes,INS56239,CarePlus,SRV101,MRI Scan,Orthopedics,S620c036,1750.61,11356.34,1022.07,12378.41,Paid,2024-06-06,RCPT641666


# HIS_C finance

In [ ]:
import pandas as pd
import requests
from io import StringIO

his_c_csv_url = "https://www.dropbox.com/scl/fi/nnftfq54u0jgtjwvsxhnl/finance.csv?rlkey=1yoduespvb04rpt2joke82z1m&st=o3z8ltdr&dl=1"

response = requests.get(his_c_csv_url)
response.raise_for_status()

his_c_finance_dataframe = pd.read_csv(StringIO(response.text))

print(his_c_finance_dataframe.shape, '\n',his_c_finance_dataframe.columns)

(200, 9) 
 Index(['transaction_id', 'patient_id', 'category', 'amount',
       'transaction_date', 'payment_method', 'insurance_claim_id', 'remarks',
       'processed_by_staff_id'],
      dtype='object')


In [162]:
s = 0
for i in his_c_finance_dataframe.columns:
    if i in his_a_finance_dataframe.columns:
        s += 1

    else:
        print(i,end=" | ")

if s == his_c_finance_dataframe.shape[1]:
    print('Ok')

else:
    print('\n\nCareful')

category | transaction_date | insurance_claim_id | remarks | processed_by_staff_id | 

Careful


In [164]:
his_c_finance_dataframe.head(3)

,transaction_id,patient_id,category,amount,transaction_date,payment_method,insurance_claim_id,remarks,processed_by_staff_id
0,9841795f,abca791b,Treatment,1302.49,2025-03-10,Card,CLM3117,The top money reveal.,f6bc8c66
1,bb0b24a9,5a31a696,Medication,3179.18,2023-05-05,UPI,CLM3147,Be poor off.,84f4be85
2,39a24247,862cfd40,Medication,1869.08,2023-10-24,Insurance,CLM8030,Want possible experience place.,a3f5600c


In [171]:
his_c_finance_dataframe.rename(columns={
    'transaction_date': 'billing_date',
    'insurance_claim_id': 'insurance_id'
}, inplace=True)


# HIS_D finance

In [ ]:
import requests
import sqlite3
import pandas as pd
import tempfile

dropbox_url = "https://www.dropbox.com/scl/fi/rsuvzgmb60bebhh5zyxed/finance.db?rlkey=mrsfx4qexfmqx344y1a22imd6&st=vit6l5nk&dl=1"

with tempfile.NamedTemporaryFile(delete=False, suffix=".db") as tmp_db:
    response = requests.get(dropbox_url)
    response.raise_for_status()  
    tmp_db.write(response.content)
    tmp_db_path = tmp_db.name

conn = sqlite3.connect(tmp_db_path)

table_names = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", table_names)

his_d_finance_dataframe = pd.read_sql("SELECT * FROM finance", conn)

print(his_d_finance_dataframe.columns,'\n', his_d_finance_dataframe.shape)

conn.close()

Tables in DB:       name
0  finance
Index(['record_id', 'patient_id', 'treatment_cost', 'insurance_claim',
       'hospital_charges', 'miscellaneous_charges', 'tax', 'discount',
       'payment_method', 'payment_status', 'date'],
      dtype='object') 
 (200, 11)


In [168]:
t = 0
for i in his_d_finance_dataframe.columns:
    if i in his_a_finance_dataframe.columns:
        t += 1

    else:
        print(i,end=" | ")

if t == his_d_finance_dataframe.shape[1]:
    print('Ok')

else:
    print('\n\nCareful')

record_id | treatment_cost | insurance_claim | hospital_charges | miscellaneous_charges | discount | date | 

Careful


In [169]:
print('HIS_A\n',his_a_finance_dataframe.head(5), '\n\n\nHIS_C\n',his_c_finance_dataframe.head(5), '\n\n\nHIS_D\n',his_d_finance_dataframe.head(5))

HIS_A
   transaction_id patient_id billing_date    amount currency payment_method  \
0       Te7aa90d   2a4cf4e4   2023-07-22  18537.49      INR            UPI   
1       T7ed7c43   37c9f9de   2025-03-13  18304.87      INR           Cash   
2       Tf4db4c9   4641840c   2024-03-27   4236.74      INR     Debit Card   
3       Te6737ec   8dd4ef1c   2025-09-07    982.60      INR     Debit Card   
4       T7003576   16707f60   2025-12-30  16698.01      INR           Cash   

  insurance_claimed insurance_id insurance_provider service_code  \
0                No     INS92486      ABC Insurance       SRV104   
1                No     INS62636           CarePlus       SRV101   
2               Yes     INS64416           CarePlus       SRV102   
3               Yes     INS58250      ABC Insurance       SRV100   
4               Yes     INS24760         XYZ Health       SRV102   

  service_description   department  staff_id  discount_applied  net_amount  \
0             Surgery    Emergency  S

In [ ]:
his_d_finance_dataframe.rename(columns={
    'record_id': 'transaction_id',
    'discount': 'discount_applied',
    'date': 'billing_date'
}, inplace=True)

his_d_finance_dataframe['amount'] = (
    his_d_finance_dataframe['treatment_cost'] +
    his_d_finance_dataframe['hospital_charges'] +
    his_d_finance_dataframe['miscellaneous_charges'] +
    his_d_finance_dataframe['tax'] -
    his_d_finance_dataframe['discount_applied']
)


# Assembling Finance Data

In [ ]:
his_a_finance_dataframe['HIS'] = 'HIS_A'
his_b_finance_dataframe['HIS'] = 'HIS_B'
his_c_finance_dataframe['HIS'] = 'HIS_C'
his_d_finance_dataframe['HIS'] = 'HIS_D'


In [177]:
for data in [his_a_finance_dataframe, his_b_finance_dataframe, his_c_finance_dataframe, his_d_finance_dataframe]:
    print(data.iloc[0],'\n\n\n')

HIS                            HIS_A
transaction_id              Te7aa90d
patient_id                  2a4cf4e4
billing_date              2023-07-22
amount                      18537.49
currency                         INR
payment_method                   UPI
insurance_claimed                 No
insurance_id                INS92486
insurance_provider     ABC Insurance
service_code                  SRV104
service_description          Surgery
department                 Emergency
staff_id                    Sa6b9ca3
discount_applied              217.02
net_amount                  18320.47
tax                          1648.84
total_amount                19969.31
payment_status                  Paid
due_date                  2026-12-31
receipt_number            RCPT309436
Name: 0, dtype: object 



HIS                            HIS_B
transaction_id              T5381df6
patient_id                  cc594dd8
billing_date              2025-01-17
amount                        497.95
currency   

In [ ]:
finance_dfs = {
    "HIS_A": his_a_finance_dataframe,
    "HIS_B": his_b_finance_dataframe,
    "HIS_C": his_c_finance_dataframe,
    "HIS_D": his_d_finance_dataframe
}

for his_name, df in finance_dfs.items():
    if 'HIS' in df.columns:
        df.drop(columns=['HIS'], inplace=True)
    
    df.insert(0, 'HIS', his_name)

In [ ]:
import pandas as pd

assembled_finance_dataframe = pd.concat(
    [his_a_finance_dataframe, his_b_finance_dataframe, his_c_finance_dataframe, his_d_finance_dataframe],
    axis=0,
    ignore_index=True,
    sort=False  
)


In [179]:
print(assembled_finance_dataframe.shape)

(880, 28)


In [180]:
assembled_finance_dataframe.head(3)

,HIS,transaction_id,patient_id,billing_date,amount,currency,payment_method,insurance_claimed,insurance_id,insurance_provider,...,payment_status,due_date,receipt_number,category,remarks,processed_by_staff_id,treatment_cost,insurance_claim,hospital_charges,miscellaneous_charges
0,HIS_A,Te7aa90d,2a4cf4e4,2023-07-22,18537.49,INR,UPI,No,INS92486,ABC Insurance,...,Paid,2026-12-31,RCPT309436,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HIS_A,T7ed7c43,37c9f9de,2025-03-13,18304.87,INR,Cash,No,INS62636,CarePlus,...,Pending,2024-10-15,RCPT721169,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HIS_A,Tf4db4c9,4641840c,2024-03-27,4236.74,INR,Debit Card,Yes,INS64416,CarePlus,...,Unpaid,2026-02-25,RCPT452362,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
assembled_finance_dataframe.to_csv('assembled_finance_dataframe.csv')

In [ ]:
import sqlite3
import os

os.makedirs('../UNIFIED_DATA', exist_ok=True)

db_path = '../UNIFIED_DATA/unified_finance.db'

conn = sqlite3.connect(db_path)
assembled_finance_dataframe.to_sql('finance', conn, if_exists='replace', index=False)
conn.close()

print('Assembled Finance data saved in UNIFIED_DATA directory !!')

Assembled Finance data saved in UNIFIED_DATA directory !!


In [188]:
print(his_a_patients_dataframe.iloc[1])

HIS                                             HIS_A
patient_id                                   dfd520ae
first_name                                     Sherri
last_name                                       Smith
dob                                        1994-04-02
gender                                           Male
blood_group                                        A-
height_cm                                         161
weight_kg                                          98
country                                         India
state                                     Maharashtra
city                                           Nanded
postal_code                                     38849
phone                                   0936410597774
email                         martinjames@example.com
emergency_contact_name                   Nathan Miles
emergency_contact_relation                     Friend
emergency_contact_phone                 9652442546708
insurance_id                

In [ ]:
import os

output_dir = r"D:\Projects\Unified Platform\UNIFIED_DATA"
os.makedirs(output_dir, exist_ok=True)

assembled_finance_dataframe.to_json(
    os.path.join(output_dir, "assembled_finance_dataframe.json"),
    orient="records",
    lines=True  
)

print("assembled_finance_dataframe saved as JSON in UNIFIED_DATA folder.")


assembled_finance_dataframe saved as JSON in UNIFIED_DATA folder.


In [ ]:
import pandas as pd
import os

# Define directory path
unified_data_dir = r"D:\Projects\Unified Platform\UNIFIED_DATA"

# Create directory if it doesn't exist
os.makedirs(unified_data_dir, exist_ok=True)

# Save assembled patients dataframe as JSON
assembled_patients_dataframe.to_json(
    os.path.join(unified_data_dir, "assembled_patients.json"),
    orient="records",
    force_ascii=False
)

# Save assembled staff dataframe as JSON
assembled_staff_dataframe.to_json(
    os.path.join(unified_data_dir, "assembled_staff.json"),
    orient="records",
    force_ascii=False
)

print("Assembled patients and staff data saved as JSON in UNIFIED_DATA directory.")

Assembled patients and staff data saved as JSON in UNIFIED_DATA directory.


In [ ]:
assembled_finance_dataframe.to_json(
    r"D:\Projects\Unified Platform\UNIFIED_DATA/assembled_finance_dataframe.json", 
    orient="records", 
    force_ascii=False
)

In [190]:
import sqlite3

conn = sqlite3.connect(r"D:\Projects\Unified Platform\UNIFIED_DATA\unified_patients.db")
cursor = conn.cursor()

cursor.execute("SELECT patient_id FROM patients LIMIT 10")
rows = cursor.fetchall()

print("First 10 patient_id values:")
for r in rows:
    print(r[0])

conn.close()

First 10 patient_id values:
81f91df9
dfd520ae
f6173246
79326454
59978f9f
b7ac3bd4
0732f5f2
24676ba0
b320ac74
7af7e665


In [198]:
import json
import os

# Path to the assembled patients JSON
unified_data_dir = r"D:\Projects\Unified Platform\UNIFIED_DATA"
patients_json_file = os.path.join(unified_data_dir, "assembled_patients.json")

# Load JSON
with open(patients_json_file, "r", encoding="utf-8") as f:
    patients_data = json.load(f)

# Print top 5 patient_ids
for row in patients_data[:5]:
    print(row.get("patient_id"))


81f91df9
dfd520ae
f6173246
79326454
59978f9f
